In [190]:
import pandas as pd
import scipy.constants
from scipy.optimize import curve_fit
from scipy.integrate import quad
from scipy.signal import argrelextrema
from sklearn.metrics import r2_score
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.transforms as transforms
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
import datetime as dt
pd.set_option('display.max_rows', 100) 


## Reading in the data and preparing DataFrames

#### MID

In [191]:
#read in the data
mid = pd.read_csv("MID.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
mid.drop([0,1], inplace=True)
print(mid.describe())
#bring the time data into a new df
time = pd.DataFrame(mid.iloc[:,0::2]) #- get every second column
print(time.describe())
#add elapsed time columns
def elapsed_time(df):
    itr = 0
    counter = len(df.columns)
    masses = [2,4,5.5,12,15,16,18,28,32,40,44]
    for i in df:
        timestamp = pd.to_datetime(df[i], format="%Y/%m/%d %H:%M:%S.%f")
        runtime = (timestamp-timestamp[2]).dt.total_seconds()
        colname = "Elapsed time, mass {}".format(masses[itr]) 
        df.insert(counter, colname, runtime)
        itr += 1
        counter += 1
    return(df)
elapsed_time(time)

time_sliced = time.iloc[:,11:].fillna(0)
print(time_sliced)

#convert elapsed times in time-dataframe back to a timestamp
def h2_time(df1,df2):
    startdate = pd.to_datetime(df1.iloc[2,0],format="%Y/%m/%d %H:%M:%S.%f")
    dic= {}
    itr = 0
    masses = [2,4,5.5,12,15,16,18,28,32,40,44]
    key = "Timestamp, mass {}".format(masses[itr])
    for label,content in df2.items():
        key = "Timestamp, mass {}".format(masses[itr])
        #.strftime('%d-%m-%Y %H:%M:%S') not used
        for row in content:
            dic.setdefault(key,[]).append((startdate+dt.timedelta(milliseconds=row*1000)))
        itr += 1
    new_df = pd.DataFrame.from_dict(dic)
    return(new_df)

new_df = h2_time(time,time_sliced)
new_df


       mass 2.0_value  mass 4.0_value  mass 5.5_value  mass 12.0_value  \
count    4.750000e+02    4.750000e+02    4.750000e+02     4.750000e+02   
mean     6.074262e-11    7.404977e-13    1.090263e-13     3.230105e-13   
std      5.055042e-12    1.499339e-13    2.920879e-14     1.572528e-14   
min      5.738620e-11    2.220000e-14    4.800000e-15     2.494000e-13   
25%      5.897690e-11    7.666500e-13    1.080000e-13     3.125500e-13   
50%      6.004630e-11    7.731000e-13    1.165000e-13     3.227000e-13   
75%      6.118575e-11    7.801500e-13    1.244500e-13     3.336500e-13   
max      9.361100e-11    8.057000e-13    1.537000e-13     3.648000e-13   

       mass 15.0_value  mass 16.0_value  mass 18.0_value  mass 28.0_value  \
count     4.750000e+02     4.750000e+02     4.750000e+02     4.750000e+02   
mean      1.894705e-13     6.075733e-13     1.350917e-12     1.924619e-12   
std       1.176345e-14     1.970131e-14     3.097804e-14     7.769109e-14   
min       1.021000e-13   

,"Timestamp, mass 2","Timestamp, mass 4","Timestamp, mass 5.5","Timestamp, mass 12","Timestamp, mass 15","Timestamp, mass 16","Timestamp, mass 18","Timestamp, mass 28","Timestamp, mass 32","Timestamp, mass 40","Timestamp, mass 44"
0,2023-01-12 17:14:44.017,2023-01-12 17:14:44.017,2023-01-12 17:14:44.017,2023-01-12 17:14:44.017,2023-01-12 17:14:44.017,2023-01-12 17:14:44.017,2023-01-12 17:14:44.017,2023-01-12 17:14:44.017,2023-01-12 17:14:44.017,2023-01-12 17:14:44.017,2023-01-12 17:14:44.017
1,2023-01-12 17:14:50.342,2023-01-12 17:14:50.342,2023-01-12 17:14:50.342,2023-01-12 17:14:50.342,2023-01-12 17:14:50.342,2023-01-12 17:14:50.342,2023-01-12 17:14:50.342,2023-01-12 17:14:50.342,2023-01-12 17:14:50.342,2023-01-12 17:14:49.591,2023-01-12 17:14:44.017
2,2023-01-12 17:14:56.667,2023-01-12 17:14:56.667,2023-01-12 17:14:56.667,2023-01-12 17:14:56.667,2023-01-12 17:14:56.667,2023-01-12 17:14:56.667,2023-01-12 17:14:56.667,2023-01-12 17:14:56.667,2023-01-12 17:14:56.667,2023-01-12 17:14:55.916,2023-01-12 17:14:44.017
3,2023-01-12 17:15:02.992,2023-01-12 17:15:02.992,2023-01-12 17:15:02.992,2023-01-12 17:15:02.992,2023-01-12 17:15:02.992,2023-01-12 17:15:02.992,2023-01-12 17:15:02.992,2023-01-12 17:15:02.992,2023-01-12 17:15:02.992,2023-01-12 17:15:02.241,2023-01-12 17:14:44.017
4,2023-01-12 17:15:09.317,2023-01-12 17:15:09.317,2023-01-12 17:15:09.317,2023-01-12 17:15:09.317,2023-01-12 17:15:09.317,2023-01-12 17:15:09.317,2023-01-12 17:15:09.317,2023-01-12 17:15:09.317,2023-01-12 17:15:09.317,2023-01-12 17:15:08.566,2023-01-12 17:14:44.017
...,...,...,...,...,...,...,...,...,...,...,...
470,2023-01-12 18:04:19.769,2023-01-12 18:04:16.768,2023-01-12 18:04:16.768,2023-01-12 18:04:16.768,2023-01-12 18:04:16.767,2023-01-12 18:04:16.768,2023-01-12 18:04:16.768,2023-01-12 18:04:16.769,2023-01-12 18:04:16.769,2023-01-12 18:04:16.017,2023-01-12 17:14:44.017
471,2023-01-12 18:04:26.094,2023-01-12 18:04:23.093,2023-01-12 18:04:23.093,2023-01-12 18:04:23.093,2023-01-12 18:04:23.092,2023-01-12 18:04:23.093,2023-01-12 18:04:23.093,2023-01-12 18:04:23.093,2023-01-12 18:04:23.094,2023-01-12 18:04:22.342,2023-01-12 17:14:44.017
472,2023-01-12 18:04:32.419,2023-01-12 18:04:29.418,2023-01-12 18:04:29.418,2023-01-12 18:04:29.418,2023-01-12 18:04:29.417,2023-01-12 18:04:29.418,2023-01-12 18:04:29.418,2023-01-12 18:04:29.418,2023-01-12 18:04:29.419,2023-01-12 18:04:28.667,2023-01-12 17:14:44.017
473,2023-01-12 18:04:38.744,2023-01-12 18:04:35.743,2023-01-12 18:04:35.743,2023-01-12 18:04:35.743,2023-01-12 18:04:35.742,2023-01-12 18:04:35.743,2023-01-12 18:04:35.743,2023-01-12 18:04:35.743,2023-01-12 18:04:35.744,2023-01-12 18:04:34.992,2023-01-12 17:14:44.017


#### Vaclogger

In [192]:
#reading the Emission current, pressures
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()
#adding an elapsed time column
#making the column into a datetime object
vac_timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(vac_timestamps-vac_timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
#drop index column (not necessary)
# vaclog.drop(index=vaclog.index[0], 
#         axis=0, 
#         inplace=True)
vaclog["Time"] = vac_timestamps.dt.strftime('%d-%m-%Y %H:%M:%S')
vaclog["Time"] = pd.to_datetime(vaclog["Time"],format='%d-%m-%Y %H:%M:%S')

vaclog.head()

,Live comments,Time,Elapsed time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
0,NaN,2023-01-12 14:57:04,0.0,-0.018483,1.900000e-09,1.000000e-09,4.996000e-09,4.528000e-07,0.000569,235.0,-5.163,4249.253,NaN,NaN
1,NaN,2023-01-12 14:57:11,7.0,-0.014883,1.900000e-09,1.000000e-09,4.996000e-09,4.533000e-07,0.000572,235.0,-5.163,4249.702,NaN,NaN
2,NaN,2023-01-12 14:57:18,14.0,-0.015426,1.900000e-09,9.970000e-10,4.996000e-09,4.533000e-07,0.000571,235.0,-5.163,4250.039,NaN,NaN
3,NaN,2023-01-12 14:57:26,22.0,-0.014883,1.890000e-09,9.890000e-10,4.996000e-09,4.533000e-07,0.000606,235.0,-5.050,4250.264,NaN,NaN
4,NaN,2023-01-12 14:57:33,29.0,-0.017521,1.890000e-09,9.920000e-10,4.996000e-09,4.533000e-07,0.000569,234.0,-5.275,4250.039,NaN,NaN


#### HiVolta log

In [193]:
#Reading in the HiVolta measurement file 
hv = pd.read_csv("hv log", sep=",")
#hv.drop(columns=["Vmon1","Vmon2","Vmon3","Vmon4","Vmon5","Vmon6","Vmon7","Vmon8","Imon3","Imon4","Imon5","Imon6","Imon7","Imon8","Comments1"], inplace=True)
hv["hv_grid"]=[element * 1e-6 for element in hv["IMon1"]]
hv["em"]=[element * 1e-6 for element in hv["IMon2"]]
hv.drop(hv.columns[hv.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
hv["em"]=abs(hv["em"])
hv["hv_grid"]=abs(hv["hv_grid"])

#adding a datetime column
datetime = pd.to_datetime(hv['Date'] + ' ' + hv['Time'],format="%d/%m/%Y %H:%M:%S.%f")
datetime = datetime.dt.strftime('%d-%m-%Y %H:%M:%S')
datetime = pd.to_datetime(datetime, format='%d-%m-%Y %H:%M:%S')
hv.insert(0,"datetime",datetime)
    
#making the column into a datetime object
hv_timestamps=pd.to_datetime(hv["Time"],format="%H:%M:%S.%f")
runtime=(hv_timestamps-hv_timestamps[0]).dt.total_seconds()    
hv.insert(2,"Elapsed time",runtime)

hv.head()



,datetime,Date,Elapsed time,Time,VMon1,VMon2,VMon3,VMon4,VMon5,VMon6,...,IMon2,IMon3,IMon4,IMon5,IMon6,IMon7,IMon8,Comment,hv_grid,em
0,2023-01-12 17:14:22,12/01/2023,0.000,17:14:22.360,0.0,0.08,0.20,0.18,0.08,0.02,...,0.0069,0.0242,-0.0522,-0.0288,0.0401,-0.0226,-0.0159,NaN,3.200000e-09,6.900000e-09
1,2023-01-12 17:14:23,12/01/2023,1.182,17:14:23.542,0.0,0.08,0.26,0.14,0.10,0.06,...,0.0069,0.0206,0.0239,0.0256,0.0314,-0.0226,-0.0159,NaN,3.100000e-09,6.900000e-09
2,2023-01-12 17:14:24,12/01/2023,2.366,17:14:24.726,0.0,0.08,0.18,0.16,0.14,0.08,...,0.0069,0.0165,0.0214,-0.0388,-0.0404,-0.0226,-0.0159,NaN,3.100000e-09,6.900000e-09
3,2023-01-12 17:14:25,12/01/2023,3.546,17:14:25.906,0.0,0.08,0.22,0.16,0.08,0.04,...,0.0073,0.0165,-0.0251,-0.0225,0.0280,-0.0226,-0.0159,NaN,2.700000e-09,7.300000e-09
4,2023-01-12 17:14:27,12/01/2023,4.719,17:14:27.079,0.0,0.08,0.24,0.14,0.06,0.06,...,0.0077,0.0216,0.0429,-0.0046,0.0206,-0.0226,-0.0159,NaN,2.600000e-09,7.700000e-09


## Data processing

In [194]:
#print vaclog comments
print(pd.unique(vaclog["Live comments"]))

[nan 'closed turbo valve' 'starting injection' 'injection complete'
 'restarted pumping']


In [195]:
#print hv comments
print(pd.unique(hv["Comment"]))
hv_comm = hv.loc[:,["datetime","Comment"]]
hv_comm.dropna(inplace=True)
print(hv_comm)
hv_comm.index

[nan 'no electron activity' 'electron activity grid repelling'
 'grid transparent' 'grid repelling']
                datetime                           Comment
9    2023-01-12 17:14:32              no electron activity
10   2023-01-12 17:14:34              no electron activity
11   2023-01-12 17:14:35              no electron activity
12   2023-01-12 17:14:36              no electron activity
293  2023-01-12 17:20:12  electron activity grid repelling
294  2023-01-12 17:20:14  electron activity grid repelling
295  2023-01-12 17:20:15  electron activity grid repelling
296  2023-01-12 17:20:16  electron activity grid repelling
481  2023-01-12 17:23:58                  grid transparent
482  2023-01-12 17:23:59                  grid transparent
483  2023-01-12 17:24:00                  grid transparent
484  2023-01-12 17:24:01                  grid transparent
719  2023-01-12 17:28:42                    grid repelling
720  2023-01-12 17:28:43                    grid repelling
721  2023-01-1

Int64Index([   9,   10,   11,   12,  293,  294,  295,  296,  481,  482,  483,
             484,  719,  720,  721,  722,  819,  820,  821,  822, 1034, 1035,
            1036, 1037, 1038, 1287, 1288, 1289, 1290, 1291, 1450, 1451, 1452,
            1453, 1898, 1899, 1900, 1901, 2092, 2093, 2094, 2095],
           dtype='int64')

#### CernOx temperature R-T conversion

In [196]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]

#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


## Checking injection volume pressure drop linearity

In [197]:
#defining a function for the straight line
def fit(x,a,b):
    return a*x + b
#curve fit for the data
y_val = vaclog["injection 1mbar"].values
x_val = (vaclog["Elapsed time"]/3600).values
params, cov = curve_fit(fit,x_val,y_val)
#straight line parameters
a, b = params
print(f"Fitted line: y = {a:.2f}*x+{b:.2f}")

Fitted line: y = -0.02*x+0.26


#### Data preparation for calculating max desorption yield (w/o recycling)

In [198]:
#Filtering background signal

#Find the timestamp where emission was turned on and off
#print("Result 1:",hv.loc[hv["Comment"]=="grid transparent"])
#print("Result 2:", hv.loc[hv["Comment"]=="grid repelling"])
#print("Result 3:", hv.loc[hv["Comment"]=="electron activity grid repelling"])

x=pd.Timestamp(2023, 1, 12, 17 ,23, 58)
y=pd.Timestamp(2023, 1, 12, 17, 28, 43)
z=pd.Timestamp(2023, 1, 12, 17, 20, 13)

#then find the time value closest to it in the MID scan file 
#print("\n","Result 4:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - x).abs().argsort(),:])
#print("\n","Result 5:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - y).abs().argsort(),:])
#print("\n","Result 6:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - z).abs().argsort(),:])

#take a mass with a signal known to be in the electronics background
elec_bck = np.mean(mid["mass 5.5_value"])

#apply elec background correction to h2 data
mid["mass 2, corr"] = mid["mass 2.0_value"] - elec_bck

#apply base pressure correction to mass 2.0 data
dyn_bck = np.mean(mid["mass 2, corr"][12:56])
print("dyn_ bck",f"{dyn_bck}")

#subtracting dyn bck like this deletes useful data
mid["mass 2, corr"] = mid["mass 2, corr"] - dyn_bck

#remove values below 0:
mid["mass 2, corr"].clip(0, inplace=True) 

#From the signal, get the partial pressures 
h2_sens = 2.13e-2 #A/mbar
h2_equiv = 2.49 #H2 equivalent pressures from N2 reading

#account for the thermal transpiration factor
T2 = 294
T1 = 4.2
T_coef = np.sqrt(T2/T1)

#apply correction factors 
p_h2 = (mid["mass 2, corr"][91:136]*h2_equiv)/(h2_sens*T_coef) 
mid.insert(22,"p_h2",p_h2)
#print(p_h2)

#baseline and emission pp for partial pressure delta
em_h2 = mid["p_h2"][91:136] #hardcoded values for "esd on"
base_h2 = mid["p_h2"][12:56] #hardcoded values for "esd off"
print(base_h2)

dyn_ bck 6.072198504784689e-11
14   NaN
15   NaN
16   NaN
17   NaN
18   NaN
19   NaN
20   NaN
21   NaN
22   NaN
23   NaN
24   NaN
25   NaN
26   NaN
27   NaN
28   NaN
29   NaN
30   NaN
31   NaN
32   NaN
33   NaN
34   NaN
35   NaN
36   NaN
37   NaN
38   NaN
39   NaN
40   NaN
41   NaN
42   NaN
43   NaN
44   NaN
45   NaN
46   NaN
47   NaN
48   NaN
49   NaN
50   NaN
51   NaN
52   NaN
53   NaN
54   NaN
55   NaN
56   NaN
57   NaN
Name: p_h2, dtype: float64


In [199]:
#NOT USED!

# #non hard coded values for "esd on/off"
# #find the local maxima
# print(mid.p_h2.values)
# mid['ESD max'] = mid.iloc[argrelextrema(mid.p_h2.values, np.greater_equal, order=5)[0]]['p_h2']
# mid['ESD min'] = mid.iloc[argrelextrema(mid.p_h2.values, np.less_equal, order=5)[0]]['p_h2']

# print(mid["ESD max"])
# print(mid["ESD min"])
# #take the 10 next values from max - define as "em on"


# #take the 10 next values from min - define as "em off"

# Apparent desorption yield calculation for H2
### Formula for the cryo setup

$$
\eta_{app}=\frac{C_{comb}\cdot (p_{n,EM} - \overline{p}_{base}) \cdot q_e}{k_B\cdot T \cdot I_{n}}
$$

T here is @RT and therefore thermal transpiration compensation should be applied afterwards to $\Delta p$.

$C_{comb}$ takes into account the conductance from the sample via the copper tube for H2 and the pumping speed for turbo pump

In [200]:
#calculation of delta_p-s
dp1 = (np.mean(p_h2)-np.mean(base_h2))*100
print(dp1)

#Constants
k_B = scipy.constants.value("Boltzmann constant")
T = 4.2
q_e = scipy.constants.value("elementary charge")
c_h2 = 37.91/1000
s_h2 = 42/1000
x_h2 = 1/((1/c_h2)+(1/s_h2))
print(x_h2)

#Emission current
I_em = np.nanmean(hv["em"].nlargest(30))
print(I_em)


nan
0.01992516581153798
3.256176666666667e-06


In [201]:
#Desorption yield
eta_h2 = (x_h2*dp1*q_e)/(k_B*T*I_em)
#eta_co = (c_co*(dp1-dp2)*q_e)/(k_B*T*I_em)
print("The desorption yield for H2 [mol/el]:",eta_h2,"\n",)#""The desorption yield for CO2 [mol/el]:",eta_co2)

The desorption yield for H2 [mol/el]: nan 



#### Data manipulation for point-by-point ESD

In [202]:
#make a sub-df from mid that includes only mass 2 data
h2 = mid.loc[:, ["mass 2.0_time", "mass 2.0_value"]]
#rename columns
h2 = h2.rename(columns={"mass 2.0_time": "datetime","mass 2.0_value": "Ion current"}).reset_index(drop=True)

print(h2)

                    datetime   Ion current
0    2023/01/12 17:14:31.367  9.361100e-11
1    2023/01/12 17:14:37.692  9.281010e-11
2    2023/01/12 17:14:44.017  9.198780e-11
3    2023/01/12 17:14:50.342  9.152550e-11
4    2023/01/12 17:14:56.667  9.081060e-11
..                       ...           ...
470  2023/01/12 18:04:07.119  5.774840e-11
471  2023/01/12 18:04:13.444  5.801880e-11
472  2023/01/12 18:04:19.769  5.815160e-11
473  2023/01/12 18:04:26.094  5.772000e-11
474  2023/01/12 18:04:32.419  5.771570e-11

[475 rows x 2 columns]


In [203]:
# Convert the 'time' column to datetime index for both dataframes
hv.index = (pd.to_datetime(hv['datetime']).dt.round(freq="S"))
h2.index = (pd.to_datetime(h2['datetime']).dt.round(freq="S"))
print(hv)

# Reindex df1 to match the datetime index of df2
hv = hv.reindex(h2.index, method='nearest', limit=1)

#Drop the "datetime" columns
hv.drop('datetime', axis=1, inplace=True)
h2.drop('datetime', axis=1, inplace=True)

# Now df1 and df2 have the same datetime index and can be joined or aligned
# by joining on the index
hv_aligned = hv.merge(h2,left_index=True, right_index=True, how="inner")

print("aligned",hv_aligned)
print(hv_aligned.describe())

                               datetime        Date  Elapsed time  \
datetime                                                            
2023-01-12 17:14:22 2023-01-12 17:14:22  12/01/2023         0.000   
2023-01-12 17:14:23 2023-01-12 17:14:23  12/01/2023         1.182   
2023-01-12 17:14:24 2023-01-12 17:14:24  12/01/2023         2.366   
2023-01-12 17:14:25 2023-01-12 17:14:25  12/01/2023         3.546   
2023-01-12 17:14:27 2023-01-12 17:14:27  12/01/2023         4.719   
...                                 ...         ...           ...   
2023-01-12 18:03:51 2023-01-12 18:03:51  12/01/2023      2969.262   
2023-01-12 18:03:52 2023-01-12 18:03:52  12/01/2023      2970.438   
2023-01-12 18:03:53 2023-01-12 18:03:53  12/01/2023      2971.623   
2023-01-12 18:03:55 2023-01-12 18:03:55  12/01/2023      2972.804   
2023-01-12 18:03:56 2023-01-12 18:03:56  12/01/2023      2973.988   

                             Time  VMon1  VMon2  VMon3  VMon4  VMon5  VMon6  \
datetime               

In [204]:
#Filtering background signal for interpolated datasets

#take a mass with a signal known to be in the background
elec_bck = np.mean(mid["mass 5.5_value"])

#apply base pressure correction to the rest of the mass spectrum
hv_aligned["mass 2 corr"] = hv_aligned["Ion current"]-elec_bck

#From the signal, get the partial pressures 
h2_sens = 2.13e-2 #A/mbar
h2_equiv = 0.46 #H2 equivalent pressures from N2 reading
p_h2_ip = hv_aligned["mass 2 corr"]/(h2_sens*h2_equiv) 

#account for the thermal transpiration factor
T2 = 4.2
T1 = 296.5
T_coef = np.sqrt(T2/T1)
p_h2_ip = p_h2_ip*T_coef 

#insert partial pressures into the dataframe
hv_aligned.insert(22,"hydrogen pp",p_h2_ip)


In [205]:
hv_aligned.fillna(0,inplace=True)
hv_aligned["hydrogen pp"].describe()

count    4.750000e+02
mean     7.365261e-10
std      6.140442e-11
min      6.957552e-10
25%      7.150777e-10
50%      7.280678e-10
75%      7.419089e-10
max      1.135784e-09
Name: hydrogen pp, dtype: float64

#### Using polynomial regression, remove the pressure decrease from pumping

In [206]:
# Load the data
t = time["Elapsed time, mass 2"]
print(t.describe())
I_ion = mid["Ion current"]
print(I_ion.describe())
#The function for the ion current-time relationship is not known. 
#numpy method for creating a polynomial model    
model = np.poly1d(np.polyfit(t, I_ion, 8))
print(model(t))

rsquared = r2_score(I_ion,model(t))
print(rsquared)

# Print the original function
plt.figure(figsize=(12,6))
plt.plot(t, I_ion)

# Add labels and title
plt.xlabel('time (s)')
plt.ylabel('Ion current')
plt.title('Ion current vs time')

plt.yscale("log")
plt.show()


count     475.000000
mean     1501.951158
std       868.329110
min         0.000000
25%       752.514500
50%      1502.027000
75%      2251.539500
max      3001.052000
Name: Elapsed time, mass 2, dtype: float64


KeyError: 'Ion current'

In [ ]:
#normalise measured data to fit
x = np.linspace(5095.951000, 19700, 3095)
I_norm = I_ion/model(x)

# Plot the normalised ion current
plt.figure(figsize=(12,6))
plt.plot(t, I_norm)

# Add labels and title
plt.xlabel('time (s)')
plt.ylabel('normalised ion current (a.u.)')
plt.title('Ion current vs time')

plt.yscale("linear")
plt.show()

#now we have 

#### Checking injection volume pressure drop linearity

In [ ]:
#defining a function for the straight line
def fit(x,a,b):
    return a*x + b
#curve fit for the data
y_val = vaclog["injection 1mbar"].values
x_val = (vaclog["Elapsed time"]/3600).values
params, cov = curve_fit(fit,x_val,y_val)
#straight line parameters
a, b = params
print(f"Fitted line: y = {a:.2f}*x+{b:.2f}")

Fitted line: y = -0.02*x+0.26


## Desorption yield point-by-point

$$
\eta_{app}=\frac{C_{comb}\cdot (p_{n,EM} - \overline{p}_{base}) \cdot q_e}{k_B\cdot T \cdot I_{n}}
$$

In [ ]:
#partial pressure delta
#wrong methodology
p_delta = 
p_delta = p_delta.clip(lower=0)
print(p_delta)
#hv_shifted = hv_aligned.shift(periods=1)
#print(hv_shifted)
#p_delta = hv_aligned["hydrogen pp"] - hv_shifted["hydrogen pp"]
#print(hv_aligned["hydrogen pp"].describe())
#print(hv_shifted["hydrogen pp"].describe())
print(p_delta.describe())

In [ ]:
#constants are already defined
#emission current and H2 partial pressures are now available at the same sampling rates
print(hv_aligned.describe())
I_n = hv_aligned["em"].replace(0, np.nan).dropna()

I_n.where(hv_aligned["em"] >= 5e-7, np.nan, inplace=True)
#pd.df.where(cond= "Where cond is True, keep the original value. Otherwise, replace with NaN")
print(I_n.describe())

eta = ((x_h2*p_delta*q_e)/(k_B*T*I_n)).dropna()
eta = eta.loc[eta.index[1]:]
print("desorption yields point-by-point:","\n", eta)

In [ ]:
#slicing eta to match hv_aligned["Elapsed time"]:
# Slice x using the index of y
hv_downsampled = hv_aligned.loc[eta.index, :]

## Electron dose


Find:

$$
Q = \int_{t_1}^{t_2} I \,dt
$$

Where $t_2$ and $t_1$ are the timestamps at which an emission current reading was taken


In [ ]:
#Integrating the emission current
def integrand(t, i):
    return i*t

i = I_n.dropna().reset_index() 

#for testing:
#t2_2 = (hv_aligned["Elapsed time"].diff(periods=1).fillna(0))

#adding an elapsed time column
timestamp3=[]
for j in hv["Time"]:
    timestamp3.append(j)
    
#making the column into a datetime object
timestamps3=pd.to_datetime(i["datetime"],format="%H:%M:%S")
runtime3=(timestamps3-timestamps3[0]).dt.total_seconds()    
i.insert(2,"Elapsed time",runtime3)

#define the n, n-1 difference in a variable "t2"
t2 = (i["Elapsed time"].diff(periods=1).fillna(0))

#replace all values from t2 that are further away than 50% of the median of the sampling rate (.. > 9) with the median value
t2.where(t2 <= 12,6,inplace=True)
print(t2)

#integration
ar = []
for x in t2:
    itr = 0
    I = quad(integrand, 0, x, args=(i["em"][itr]))
    ar.append(I[0])
    itr += 1
    #if previous value was not NaN, replace the value with the median of elapsed time
    #if 
    

In [ ]:
#calculate the cumulative charge
Q_cumul = np.cumsum(ar)
Q_sum = sum(ar)
print("Total charge emitted from EGA during ~5000 seconds:", f"{Q_sum:.4E}", "C")

#cumulative nr of electrons emitted
N_e = Q_sum/q_e
Ne_cumul = Q_cumul/q_e
print("Total number of electron emitted during ~5000 seconds:", f"{N_e:.4E}", "e")
      
#To get the electron dose, divide n of electrons by sample surface area of 276 cm2, multiply by grid transmittance of 0.9
tot_dose = (N_e/276)*0.9
cumul_dose = (Ne_cumul/276)*0.9
print("Total electron dose on the sample ~5000 seconds:", f"{tot_dose:.4E}", "e-/cm^2")
      
#add these to the "i" dataframe:
i.insert(3,"Cumulative charge", Q_cumul)
i.insert(4,"Cumulative n. of electrons emitted from EGA", Ne_cumul)
i.insert(5,"Cumulative electron dose on the HL sample", cumul_dose)
i.set_index("datetime", drop=True, inplace=True)

#Ignore the first value, also, match the shape of eta
i = i.loc[i.index[1]:]
print(i)


# Plotting

In [ ]:
#define parameters for plotting
#annotation params
font = dict(size = "x-small", color ="green", style ="italic",rotation="vertical")


In [ ]:
#plot the aligned dataframe results
plt.figure(figsize=(12,6))
plt.plot(hv_aligned["Elapsed time"],hv_aligned["Ion current"],linestyle="-",marker=".", markersize=4,label='H2 ion current')
plt.plot(hv_aligned["Elapsed time"],hv_aligned["em"],linestyle="-",marker=".", markersize=4,label='Emission current')
plt.xlabel('Elapsed time (s)')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("Current evolution 1 ML tech. surface pre-injection, EGA fully in, dynamic vacuum")


plt.yscale('log')


In [ ]:
#plot the h2 mid 
#seems to be too little rows?
plt.figure(figsize=(12,6))
plt.plot(hv_aligned["Elapsed time"],hv_aligned["Ion current"],linestyle="-",marker=".", markersize=4,label='H2 ion current')
plt.xlabel('Elapsed time (s)')
plt.ylabel('Ion current (A)')
plt.legend(loc="upper right")
plt.title("Ion current evolution of H2, 1 ML tech. surface pre-injection, EGA fully in, dynamic vacuum")


plt.yscale('linear')


In [ ]:
#Plotting the all the relevant MID spectrums
plt.figure(figsize=(14,8))

plt.plot(new_df["Timestamp, mass 2"],mid["mass 2.0_value"],marker=".", markersize=4,label='M2')
plt.plot(new_df["Timestamp, mass 4"],mid["mass 4.0_value"],marker=".", markersize=4,label='M4')
plt.plot(new_df["Timestamp, mass 12"],mid["mass 12.0_value"],marker=".", markersize=4,label='M12')
plt.plot(new_df["Timestamp, mass 15"],mid["mass 15.0_value"],marker=".", markersize=4,label='M16')
plt.plot(new_df["Timestamp, mass 16"],mid["mass 16.0_value"],marker=".", markersize=4,label='M15')
plt.plot(new_df["Timestamp, mass 18"],mid["mass 18.0_value"],marker=".", markersize=4,label='M18')
plt.plot(new_df["Timestamp, mass 28"],mid["mass 28.0_value"],marker=".", markersize=4,label='M28')
plt.plot(new_df["Timestamp, mass 32"],mid["mass 32.0_value"],marker=".", markersize=4,label='M32')
plt.plot(new_df["Timestamp, mass 40"],mid["mass 40.0_value"],marker=".", markersize=4,label='M40')
# plt.plot(new_df["Timestamp, mass 44"],mid["mass 44.0_value"],marker=".", markersize=4,label='M44')

plt.xlabel('Timestamp')
plt.ylabel('Ion current (A)')
plt.legend(title="Mol masses", loc="upper right")
plt.title("S1 MID scan")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(14))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')

#write comments
trans  = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)
for i in hv_comm.index:
    plt.text(hv_comm["datetime"][i],0.95,str(hv_comm["Comment"][i]),fontdict=font, transform=trans)


plt.savefig(r'./graphs/MID.png')
plt.show()

In [ ]:
#plotting HiVolta log
plt.figure(figsize=(12,6))
plt.plot(hv["Elapsed time"],hv["hv_grid"],marker=".", markersize=4,label='Grid current')
plt.plot(hv["Elapsed time"],hv["em"],marker=".", markersize=4,label='Emission current')
plt.xlabel('Elapsed time (s)')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("Current evolution 1 ML tech. surface pre-injection, EGA fully in, dynamic vacuum")


plt.yscale('log')


In [ ]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(12,6))
plt.plot(vaclog["Time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')
plt.xlabel('Timestamp')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("S1 Total pressure evolution")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')    
       
plt.savefig(r'./graphs/Barions.png')
plt.show()

In [ ]:
## Plotting the temp evolution
plt.figure(figsize=(10,6))
plt.plot((vaclog["Time"]),polyfit(A,vaclog["T-CERNOX"]),marker=".", markersize=5,label='T-CERNOX')          
plt.xlabel('Timestamp')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("S1 Temperature evolution")
plt.yscale('linear')
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
    
plt.savefig(r'./graphs/Temp.png')
plt.show()

In [ ]:
#checking inj pressure linearity, graph
fig, ax = plt.subplots()
y_fit = fit(x_val,a,b)
ax.plot((vaclog["Elapsed time"]/3600),vaclog["injection 1mbar"],marker=".", markersize=5,label='inj volume pressure')
ax.plot((vaclog["Elapsed time"]/3600),y_fit, "--",color="red", label="Y= -ax + b fit") 
ax.legend(loc="upper right")

ax.set_xlabel('Elapsed time (h)')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('linear')
plt.title("Injection pressure evolution 1mbar conductance gauge")

In [ ]:
#plotting point-by-point eta
plt.figure(figsize=(10,6))
plt.plot(hv_downsampled["Elapsed time"],eta,marker=".", markersize=4,label='eta')
#plt.plot(vaclog["Elapsed time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Electon stimulated desorption yield (M/el)')
plt.legend()
plt.title("ESD vs time, p_n - p_n-5")
ax = plt.gca()
ax.set_yscale('log')
ax.grid(which="major",axis="x")
plt.tick_params(axis="y", which='minor')
plt.grid(visible=None, which='both', axis='both')
plt.show()

In [ ]:
#plotting esd vs electron dose
plt.figure(figsize=(10,6))
plt.plot(i["Cumulative electron dose on the HL sample"],eta,marker=".", markersize=4,label='eta')
#plt.plot(vaclog["Elapsed time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')

plt.xlabel('Electron dose (e/cm^2)')
plt.ylabel('Electon stimulated desorption yield (M/el)')
plt.legend()
plt.title("ESD vs dose")
ax = plt.gca()
ax.set_yscale('log')
ax.grid(which="major",axis="x")
plt.tick_params(axis="y", which='minor')
plt.grid(visible=None, which='both', axis='both')
plt.show()